In [2]:
from numpy import *
from sympy import *

In [3]:
#Function to order a matrix by increasing number of zeroes before leading entry by row
def orderzero(X):
    r=(shape(X))[0]
    c=(shape(X))[1]
    N=[]
    #Creating a corresponding array for number of zeroes before leading entry in each row
    for i in range(0,r):
        n=0
        for j in range(0,c):
            if(X[i,j]==0):
                n=n+1
            if(X[i,j]!=0 or j==c-1):
                N.append(n)
                break
    #Sorting the array, and simultaneously the matrix, in increasing order of number of
    #zeros before leading entry rowwise
    for i in range(0,r):
        for j in range(0, r - i - 1):
            if N[j] > N[j + 1]:
                temp = N[j]
                N[j] = N[j+1]
                N[j+1] = temp
                X=X.elementary_row_op(op='n<->m', row1=j, row2=j+1)
    return(X)

In [4]:
#Function to reduce any matrix into row reduced echelon form
def rreform(X):
    r=(shape(X))[0]
    c=(shape(X))[1]
    X=orderzero(X)
    #u is the number of complete zero columns in the matrix
    u=0
    try:
        for i in range(0,c):
            #Flag variable initially set to 0, changes to 1 if atleast 1 non-zero element is found in the ith column
            flag=0
            if(X[i-u,i]!=0):
                flag=1
            #If the first "focus element" of ith column is zero, its row is swapped with the row of the first non-zero
            #element below it in the ith column
            else:
                for x in range(i-u+1,r):
                    if(X[x,i]!=0):
                        X=X.elementary_row_op(op='n<->m', row1=i-u, row2=x)
                        flag=1
                        break
            #If the "focus element" of the ith column is zero, there is no row operations done,
            #and the "focus element" shifts one to the right in the matrix
            if(flag==0):
                u=u+1
            #If the "focus element" of the ith column is non-zero, all elements below it are converted to 0 using row operations,
            #and the "focus element" shifts one to the right and one down in the matrix
            else:
                #The "focus element" is changed to 1, to avoid any divide by zero errors
                X=X.elementary_row_op(op='n->kn', k=1/X[i-u,i], row1=i-u)
                #All elements below the "focus element" are converted to 0 using row operations
                for j in range(i-u+1,r):
                    X=X.elementary_row_op(op='n->n+km', k=-(X[j,i]), row1=j, row2=i-u)
    #In cases where the number of columns are greater than the number of rows in the matrix, the process is terminated when a
    #index error is encountered as at this point the matrix is already in row reduced echelon form
    except IndexError:
        exit
    return(X)
    

In [5]:
#Function to find the normal form of a matrix
def normalform(X):
    r=(shape(X))[0]
    c=(shape(X))[1]
    X=rreform(X)
    #Finding the leading entry of the ith row, and converting all elements to its
    #right to 0 using column operations
    for i in range(0,r):
        for j in range(0,c):
            if(X[i,j]!=0):
                for k in range(j+1,c):
                    X=X.elementary_col_op(op='n->n+km',k=-(X[i,k]),col1=k, col2=j)
    #For cases in which the identity submatrix of the normal form isn't in the top left,
    #finding all the fully zero columns in the normal form from right to left and
    #shifting it to the right most column possible such that all full zero columns are
    #stacked ont he right, and the identity submatrix is on the top left in the normal form
    for i in range(c-1,-1,-1):
        flag=0
        for j in range(0,r):
            if(X[j,i]!=0):
                flag=1
        if(flag==0):
            for k in range(i+1,c):
                X=X.elementary_col_op(op='n<->m', col1=k, col2=k-1)
    return(X)

In [6]:
#Function to find rank of a matrix
def matrixrank(X):
    rank=0
    r=(shape(X))[0]
    c=(shape(X))[1]
    X=normalform(X)
    #Finding number of rows which have a (1) element
    for i in range(0,r):
        for j in range(0,c):
            if(X[i,j]!=0):
                rank=rank+1
                break
    return(rank)

In [7]:
#Function to find inverse of a square matrix using row operations
def inverseusingops(X):
    r=(shape(X))[0]
    c=(shape(X))[1]
    #Check whether the matrix is a square matrix and whether it is invertible
    if((r==c) and (det(X)!=0)):
        n=r
        #Creating an identity matrix of same order as the input matrix
        I=eye(r)
        #Simultaneously applying row operations to both matrices, such that the input matrix is converted to an identity matrix
        #The identity matrix will convert to the inverse of the input matrix
        for i in range(0,n):
            #If a diagonal element is zero, swapping it with a row below it with a non-zero element in the same column
            if(X[i,i]==0):
                for x in range(i+1,r):
                    if(X[x,i]!=0):
                        I=I.elementary_row_op(op='n<->m', row1=i, row2=x)
                        X=X.elementary_row_op(op='n<->m', row1=i, row2=x)
                        break
            #Converting the diagonal element to 1
            I=I.elementary_row_op(op='n->kn', k=1/X[i,i], row1=i)
            X=X.elementary_row_op(op='n->kn', k=1/X[i,i], row1=i)
            for j in range(0,n):
                #Converting all remaining elements in the column to zero
                if(i!=j):
                    I=I.elementary_row_op(op='n->n+km', k=-X[j,i], row1=j, row2=i)
                    X=X.elementary_row_op(op='n->n+km', k=-X[j,i], row1=j, row2=i)
        return(I)
    else:
        return("NOT INVERTIBLE")

In [8]:
input=Matrix(mat(input("Enter matrix")))

In [9]:
ordered=orderzero(input)
rrechform=rreform(input)
normal=normalform(input)
rank=matrixrank(input)
inv=inverseusingops(input)

In [10]:
display("Input matrix is: ",input)
display("Row reduced echelon form of the matrix is: ",rrechform)
display("Normal form of the matrix is: ",normal)
display("Rank of matrix is",rank)
display("Inverse of matrix is",inv)

'Input matrix is: '

Matrix([
[0, 5,  4, 3],
[8, 6,  4, 4],
[6, 3, 56, 8],
[0, 4,  6, 7]])

'Row reduced echelon form of the matrix is: '

Matrix([
[1, 3/4,    1/2,    1/2],
[0,   1, -106/3,  -10/3],
[0,   0,      1, 59/542],
[0,   0,      0,      1]])

'Normal form of the matrix is: '

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

'Rank of matrix is'

4

'Inverse of matrix is'

Matrix([
[ -107/776, 191/1552,   1/388,   -11/776],
[ 341/1164,    5/776,  -5/582, -139/1164],
[  61/2328, -23/1552, 23/1164,  -59/2328],
[-221/1164,    7/776,  -7/582,  271/1164]])